In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf

from ngram import *
from baselines import *
from error_analysis import *

In [2]:
d = tidy_labels(pd.concat(load_cf_data()))
d = d.dropna(subset=['aggression_score'])
d = d.iloc[np.random.permutation(np.arange(d.shape[0]))]

In [3]:
max_features = 5000
architecture = [50, 20]

In [4]:
ngram_feature_pipeline = Pipeline([
    ('vect', CountVectorizer(ngram_range = (1,5), analyzer = 'char', max_features = max_features)),
    ('tfidf', TfidfTransformer(sublinear_tf=True,norm='l2')),
])

In [5]:
training_epochs = 75
batch_size = 500
display_step = 500

# Agression Model

In [6]:
labels = empirical_dist(d['aggression_score'], w = 0.25)
data = get_labeled_comments(d, labels)
train, test = train_test_split(data, test_size = 0.2, random_state=0)

y_train =train.ix[:, train.columns != 'x'].values
y_test =test.ix[:, train.columns != 'x'].values

ngram_feature_extractor = ngram_feature_pipeline.fit(train['x'])
X_train = ngram_feature_extractor.transform(train['x'])
X_test = ngram_feature_extractor.transform(test['x'])

In [7]:
classes = np.arange(y_test.shape[1])
y_train_mean = y_train.dot(classes).reshape((y_train.shape[0], 1))
y_test_mean = y_test.dot(classes).reshape((y_test.shape[0], 1))


NN_REG(X_train,
        y_train_mean,
        X_test,
        y_test_mean,
        training_epochs = 100,
        batch_size = 500,
        display_step = 1000,
        architecture = architecture)

Batch: 0001 cost= 200.453338623

	Train RMSE:  11.876830211864322
	Test RMSE:  12.0066542263117

	Train R^2:  -0.00946412740553
	Test R^2:  -0.00871802623687


Batch: 1001 cost= 51.304167497

	Train RMSE:  0.7548548177104467
	Test RMSE:  0.805283885951192

	Train R^2:  -0.655800525268
	Test R^2:  -0.518735415726


Batch: 2001 cost= 34.789401670

	Train RMSE:  0.566675978440958
	Test RMSE:  0.6050497697341211

	Train R^2:  -0.557587572695
	Test R^2:  -0.485284717808


Batch: 3001 cost= 19.636608984

	Train RMSE:  0.45018246841338794
	Test RMSE:  0.4771567124636231

	Train R^2:  0.241276617386
	Test R^2:  0.184140579519


Batch: 4001 cost= 9.769432353

	Train RMSE:  0.38627761027694735
	Test RMSE:  0.4050118334136526

	Train R^2:  0.486521315111
	Test R^2:  0.43073138452


Batch: 5001 cost= 3.949548425

	Train RMSE:  0.36048278148966106
	Test RMSE:  0.37884493483603676

	Train R^2:  0.573137239008
	Test R^2:  0.51761048325


Batch: 6001 cost= 1.525451281

	Train RMSE:  0.3466866336761824

In [10]:
sk_reg_model = Ridge(alpha = 1.0).fit(X_train, y_train.dot(classes))
_ = get_regression_metrics(sk_reg_model.predict(X_test), y_test.dot(classes))

# Random on Random

In [17]:
d_b = tidy_labels(load_cf_data()[1])
d_b = d_b.dropna(subset=['aggression_score'])

labels_b = empirical_dist(d_b['aggression_score'], w = 0.25)
data_b = get_labeled_comments(d_b, labels_b)
train_b, test_b = train_test_split(data_b, test_size = 0.2, random_state=0)

y_train_b =train_b.ix[:, train_b.columns != 'x'].values
y_test_b =test_b.ix[:, train_b.columns != 'x'].values

ngram_feature_extractor_b = ngram_feature_pipeline.fit(train_b['x'])
X_train_b = ngram_feature_extractor_b.transform(train_b['x'])
X_test_b = ngram_feature_extractor_b.transform(test_b['x'])

In [24]:
y_test_mean_b.shape

(3999, 1)

In [25]:
classes_b = np.arange(y_test_b.shape[1])
y_train_mean_b = y_train_b.dot(classes).reshape((y_train_b.shape[0], 1))
y_test_mean_b = y_test_b.dot(classes).reshape((y_test_b.shape[0], 1))


NN_REG(X_train_b,
        y_train_mean_b,
        X_test_b,
        y_test_mean_b,
        training_epochs = 100,
        batch_size = 500,
        display_step = 1000,
        architecture = architecture)

Batch: 0001 cost= 1591.054687500

	Train RMSE:  40.05427712220673
	Test RMSE:  39.97872458692088

	Train R^2:  -12.6953108298
	Test R^2:  -12.5926494438


Batch: 1001 cost= 60.303840637

	Train RMSE:  1.7028794493942203
	Test RMSE:  1.8017602730189872

	Train R^2:  -0.0240289297425
	Test R^2:  -0.0348713970643


Batch: 2001 cost= 50.808229110

	Train RMSE:  0.8433082921767926
	Test RMSE:  1.0108507614701627

	Train R^2:  -0.0835647488017
	Test R^2:  -0.0776316936593


Batch: 3001 cost= 41.912141724

	Train RMSE:  0.5287524605212611
	Test RMSE:  0.7498003358278855

	Train R^2:  -0.219931160498
	Test R^2:  -0.127767962056


Optimization Finished!


In [26]:
sk_reg_model_b = Ridge(alpha = 1.0).fit(X_train_b, y_train_b.dot(classes))
_ = get_regression_metrics(sk_reg_model.predict(X_test), y_test.dot(classes))

R^2: 0.569
RMSE: 0.367
Mean absolute error: 0.272
Median absolute error: 0.205
Pearson: 0.837
Spearman: 0.744
